<a href="https://colab.research.google.com/github/yashkapur0403/Neural-Networks-Practise/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import numpy as np
import cv2
import zipfile
from sklearn.model_selection import train_test_split

# Unzip the dataset from Google Drive
zip_path = "/content/drive/My Drive/Colab Notebooks/chest_xray.zip"
extract_path = "/content/chest_xray"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Function to load images and labels
def load_images_from_folder(folder_path, label):
    data = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))          # Resize to 64x64
            img_flat = img.flatten() / 255.0         # Normalize and flatten
            data.append((img_flat, label))
    return data

normal_data  = load_images_from_folder("/content/chest_xray/chest_xray/train/NORMAL", 0)
pneumonia_data = load_images_from_folder("/content/chest_xray/chest_xray/train/PNEUMONIA", 1)
all_data = normal_data + pneumonia_data
np.random.shuffle(all_data)

X = np.array([item[0] for item in all_data])
y = np.array([item[1] for item in all_data])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
class SimpleSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, epochs=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.epochs = epochs
        self.weights = None
        self.bias = 0

    def fit(self, X, y):
        num_samples, num_features = X.shape
        y_copy = np.where(y == 0, -1, 1)  # convert 0 to -1 for SVM

        self.weights = np.zeros(num_features)

        for epoch in range(self.epochs):
            for i in range(num_samples):
                x_i = X[i]
                condition = y_copy[i] * (np.dot(x_i, self.weights) + self.bias)

                if condition >= 1:
                    self.weights -= self.lr * self.lambda_param * self.weights
                else:
                    self.weights -= self.lr * (self.lambda_param * self.weights - y_copy[i] * x_i)
                    self.bias -= self.lr * (-y_copy[i])

            if epoch % 100 == 0:
              print(f"Epoch {epoch}/{self.epochs}")

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output >= 0, 1, 0)


svm = SimpleSVM(learning_rate=0.001, lambda_param=0.01, epochs=1000)
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

accuracy = np.mean(predictions == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 0/1000
Epoch 100/1000
Epoch 200/1000
Epoch 300/1000
Epoch 400/1000
Epoch 500/1000
Epoch 600/1000
Epoch 700/1000
Epoch 800/1000
Epoch 900/1000
Test Accuracy: 93.29%


In [16]:
# Re-run this cell (alone) to re-mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive
